# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846741443685                   -0.70    4.8
  2   -7.852308620072       -2.25       -1.53    1.0
  3   -7.852646223137       -3.47       -2.52    3.2
  4   -7.852646677461       -6.34       -3.37    2.0
  5   -7.852646686214       -8.06       -4.65    1.8
  6   -7.852646686728       -9.29       -5.55    3.0
  7   -7.852646686730      -11.76       -6.29    1.5
  8   -7.852646686730      -13.14       -7.90    2.0
  9   -7.852646686730      -14.75       -8.10    2.8
 10   -7.852646686730   +  -14.75       -8.84    2.0
 11   -7.852646686730      -14.75       -9.12    1.0
 12   -7.852646686730   +    -Inf       -8.85    1.5
 13   -7.852646686730      -14.75       -8.12    2.2
 14   -7.852646686730   +  -14.75       -7.78    3.0
 15   -7.852646686730   +  -14.75       -7.45    2.0
 16   -7.852646686730      -15.05       -7.11    3.5
 17   -7.852646686730   +    -Inf       -6.93 

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846685098963                   -0.70           4.8
  2   -7.852526147503       -2.23       -1.63   0.80    2.0
  3   -7.852636144074       -3.96       -2.73   0.80    1.0
  4   -7.852646506431       -4.98       -3.26   0.80    2.2
  5   -7.852646673860       -6.78       -4.11   0.80    1.0
  6   -7.852646686370       -7.90       -4.82   0.80    1.5
  7   -7.852646686722       -9.45       -5.64   0.80    1.8
  8   -7.852646686730      -11.12       -6.79   0.80    2.2
  9   -7.852646686730      -12.71       -7.66   0.80    2.8
 10   -7.852646686730      -14.57       -8.22   0.80    2.0
 11   -7.852646686730      -15.05       -8.70   0.80    1.8
 12   -7.852646686730   +  -14.45       -9.57   0.80    1.5
 13   -7.852646686730      -14.75       -9.99   0.80    2.5
 14   -7.852646686730   +  -14.75      -11.21   0.80    1.0
 15   -7.852646686730      -14.75      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.392138e+01     3.672561e+00
 * time: 0.43007588386535645
     1     9.210207e-01     1.754311e+00
 * time: 0.6578269004821777
     2    -1.837637e+00     1.980238e+00
 * time: 0.6829869747161865
     3    -3.864499e+00     1.955284e+00
 * time: 0.7187960147857666
     4    -5.318041e+00     1.606584e+00
 * time: 0.7552459239959717
     5    -6.863490e+00     8.896595e-01
 * time: 0.7916979789733887
     6    -7.047855e+00     1.014120e+00
 * time: 0.8167829513549805
     7    -7.462746e+00     7.150034e-01
 * time: 0.8420910835266113
     8    -7.550530e+00     6.870708e-01
 * time: 0.868290901184082
     9    -7.686539e+00     3.893028e-01
 * time: 0.8954989910125732
    10    -7.749753e+00     1.558979e-01
 * time: 0.9213120937347412
    11    -7.784597e+00     9.018434e-02
 * time: 0.9480860233306885
    12    -7.814688e+00     1.097167e-01
 * time: 0.9738979339599609
    13    -7.828429e+00     9.558315e-02
 * time: 0.9990680217

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846787651693                   -0.71    4.5
  2   -7.852289819119       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646685957                   -2.56
  2   -7.852646686730       -9.11       -5.43
  3   -7.852646686730      -14.75      -10.18
  4   -7.852646686730   +    -Inf      -16.00


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 4.994481158180154e-13
|ρ_newton - ρ_scfv| = 1.1114281970870945e-13
|ρ_newton - ρ_dm|   = 3.483645828777761e-10
